#Linear regression model from scratch

In [ ]:
import numpy as np
import torch

In [ ]:
inputs = np.array([[73, 67, 43],
[91, 88, 64],
[87, 134, 58],
[102, 43, 37],
[69, 96, 70]], dtype='float32')

In [ ]:
targets = np.array([[56],
[81],
[119],
[22],
[103]], dtype='float32')

In [ ]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [ ]:
weight = torch.randn(2, 3, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

In [ ]:
inputs @ weight.t() + bias

tensor([[ -73.3656,  -98.3297],
        [ -92.2246, -142.0584],
        [-139.1011, -222.6786],
        [ -59.5573,  -20.0133],
        [ -92.0101, -184.5744]], grad_fn=<AddBackward0>)

In [ ]:
def model(inputs):
    return inputs @ weight.t() + bias

In [ ]:
prediction = model(inputs)
print(prediction)

tensor([[ -73.3656,  -98.3297],
        [ -92.2246, -142.0584],
        [-139.1011, -222.6786],
        [ -59.5573,  -20.0133],
        [ -92.0101, -184.5744]], grad_fn=<AddBackward0>)


##Loss function

In [ ]:
diff0 = prediction - targets
torch.sum(diff0 * diff0) / diff0.numel()

tensor(43282.0078, grad_fn=<DivBackward0>)

In [ ]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [ ]:
loss = mse(prediction, targets)
print(loss)

tensor(43282.0078, grad_fn=<DivBackward0>)


##Compute gradients

In [ ]:
loss.backward()
print(weight.grad)
print(bias.grad)

tensor([[-13887.2930, -16144.9492,  -9657.4570],
        [-17083.6816, -21033.5762, -12482.7949]])
tensor([-167.4517, -209.7309])


##Train the model using gradient descent

In [ ]:
with torch.no_grad():
    weight -= weight.grad * 1e-5
    bias -= bias.grad * 1e-5
    weight.grad.zero_()
    bias.grad.zero_()

In [ ]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(21364.4121, grad_fn=<DivBackward0>)


In [ ]:
for i in range(100):
    prediction = model(inputs)
    loss = mse(prediction, targets)
    loss.backward()
    with torch.no_grad():
        weight -= weight.grad * 1e-5
        bias -= bias.grad * 1e-5
        weight.grad.zero_()
        bias.grad.zero_()

In [ ]:
prediction = model(inputs)
loss = mse(prediction, targets)
print(loss)

tensor(913.8755, grad_fn=<DivBackward0>)


In [ ]:
prediction

tensor([[62.2296, 69.7153],
        [88.4226, 85.4121],
        [96.4126, 91.2257],
        [51.4243, 93.9224],
        [94.8999, 64.9022]], grad_fn=<AddBackward0>)

In [ ]:
targets

tensor([[ 56.],
        [ 81.],
        [119.],
        [ 22.],
        [103.]])

In [1]:
import numpy as np
import torch

In [2]:
# Targets (apples, oranges)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [5]:
from torch.utils.data import TensorDataset

In [6]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [7]:
from torch.utils.data import DataLoader

In [8]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [9]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 134.,  58.],
        [ 69.,  96.,  70.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.]])
tensor([[119., 133.],
        [103., 119.],
        [ 81., 101.],
        [ 22.,  37.],
        [ 56.,  70.]])


In [10]:
import torch.nn as nn

In [11]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.2660,  0.3357,  0.3912],
        [-0.2694,  0.0776, -0.4449]], requires_grad=True)
Parameter containing:
tensor([0.0524, 0.4664], requires_grad=True)


In [12]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2660,  0.3357,  0.3912],
         [-0.2694,  0.0776, -0.4449]], requires_grad=True),
 Parameter containing:
 tensor([0.0524, 0.4664], requires_grad=True)]

In [13]:
preds = model(inputs)
preds

tensor([[ 58.7825, -33.1292],
        [ 78.8345, -45.6916],
        [ 90.8647, -38.3740],
        [ 56.0941, -40.1339],
        [ 78.0143, -41.8144]], grad_fn=<AddmmBackward0>)

In [14]:
import torch.nn.functional as F

In [15]:
loss_fn = F.mse_loss

In [16]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(9592.4736, grad_fn=<MseLossBackward0>)


In [17]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [18]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [19]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 621.0394
Epoch [20/100], Loss: 324.6448
Epoch [30/100], Loss: 286.1794
Epoch [40/100], Loss: 256.5190
Epoch [50/100], Loss: 230.3979
Epoch [60/100], Loss: 207.3136
Epoch [70/100], Loss: 186.9035
Epoch [80/100], Loss: 168.8503
Epoch [90/100], Loss: 152.8739
Epoch [100/100], Loss: 138.7280


In [20]:
prediction = model(inputs)
prediction

tensor([[ 60.9312,  73.6178],
        [ 83.5428,  96.4687],
        [109.6005, 137.1961],
        [ 42.7995,  55.1384],
        [ 91.5806, 101.3458]], grad_fn=<AddmmBackward0>)

In [21]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [22]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[58.8661, 70.3220]], grad_fn=<AddmmBackward0>)